In [2]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, BatchNormalization as BN, Dense
from tensorflow.keras.models import Sequential

In [16]:
model = Sequential()

# Convolution Layer 1
model.add(Conv2D(filters=8, kernel_size=(3,3), padding='same', activation='relu', input_shape=(180, 50, 3)))
model.add(MaxPooling2D(pool_size=(2,2)))

# Convolution Layer 2
model.add(Conv2D(filters=16, kernel_size=(3,3), padding='same', activation='relu'))
model.add(BN())
model.add(MaxPooling2D(pool_size=(2,2)))

# Convolution Layer 3
model.add(Conv2D(filters=32, kernel_size=(3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

# Convolution Layer 4
model.add(Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu'))
model.add(BN())
model.add(MaxPooling2D(pool_size=(2,2)))

# Convolution Layer 5
model.add(Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

# Convolution Layer 6
model.add(Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu'))
model.add(BN())

# Fully Connected Layer
model.add(Flatten())
model.add(Dense(221, activation='softmax'))

In [17]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_32 (Conv2D)          (None, 180, 50, 8)        224       
                                                                 
 max_pooling2d_26 (MaxPoolin  (None, 90, 25, 8)        0         
 g2D)                                                            
                                                                 
 conv2d_33 (Conv2D)          (None, 90, 25, 16)        1168      
                                                                 
 batch_normalization_16 (Bat  (None, 90, 25, 16)       64        
 chNormalization)                                                
                                                                 
 max_pooling2d_27 (MaxPoolin  (None, 45, 12, 16)       0         
 g2D)                                                            
                                                      

In [20]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])